# Predicting burned area of the forest using Support Vector Machines

#### To classify whethere the burned area of the forest is SMALL or LARGE.


## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc as py

## Importing data

In [2]:
py.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']

In [3]:
sql = """
        Select * from forestfires
"""
cnxn = py.connect('DRIVER={SQL Server Native Client 11.0};SERVER=' + 'DESKTOP-US3IT7O' ';DATABASE=' + 'Study' + ';Trusted_Connection=' 'yes;')

df = pd.read_sql(sql,cnxn)
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0,...,0,0,0,0,0,0,0,0,0,small


## Exploratory Data Analysis

### Features: 
* month - 	month of the year: 'jan' to 'dec'
* day	- day of the week: 'mon' to 'sun'
* FFMC  - FFMC index from the FWI system: 18.7 to 96.20
* DMC	- DMC index from the FWI system: 1.1 to 291.3
* DC	- DC index from the FWI system: 7.9 to 860.6
* ISI	- ISI index from the FWI system: 0.0 to 56.10
* temp	- temperature in Celsius degrees: 2.2 to 33.30
* RH	- relative humidity in %: 15.0 to 100
* wind	- wind speed in km/h: 0.40 to 9.40
* rain	- outside rain in mm/m2 : 0.0 to 6.4
* Size_Categorie --?	the burned area of the forest ( Small = 1 , Large = 0)

In [4]:
df.shape

(517, 31)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    object 
 10  area           517 non-null    object 
 11  dayfri         517 non-null    object 
 12  daymon         517 non-null    object 
 13  daysat         517 non-null    object 
 14  daysun         517 non-null    object 
 15  daythu         517 non-null    object 
 16  daytue         517 non-null    object 
 17  daywed         517 non-null    object 
 18  monthapr  

In [6]:
df = df.drop(columns=['dayfri','daymon','daysat','daysun','daythu','daytue','daywed',
                      'monthapr','monthaug','monthdec','monthfeb','monthjan','monthjul',
                      'monthjun','monthmar','monthmay','monthnov','monthoct','monthsep'], axis=1)

In [7]:
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0,6.44,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0,54.29,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0,11.16,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0,0,small


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    object 
 10  area           517 non-null    object 
 11  size_category  517 non-null    object 
dtypes: float64(6), int64(1), object(5)
memory usage: 48.6+ KB


In [9]:
df.isnull().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
size_category    0
dtype: int64

In [10]:
df.duplicated().sum()

8

In [11]:
duplicates = df[df.duplicated()]
duplicates

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
53,aug,wed,92.1,111.2,654.1,9.6,20.4,42,4.9,0,0,small
100,aug,sun,91.4,142.4,601.4,10.6,19.8,39,5.4,0,0,small
215,mar,sat,91.7,35.8,80.8,7.8,17.0,27,4.9,0,28.66,large
303,jun,fri,91.1,94.1,232.1,7.1,19.2,38,4.5,0,0,small
426,aug,thu,91.6,248.4,753.8,6.3,20.4,56,2.2,0,0,small
461,aug,sat,93.7,231.1,715.1,8.4,18.9,64,4.9,0,0,small
501,aug,tue,96.1,181.1,671.2,14.3,21.6,65,4.9,0.8,0,small
508,aug,fri,91.0,166.9,752.6,7.1,25.9,41,3.6,0,0,small


In [12]:
df = df.drop_duplicates(keep='first',ignore_index=True)
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...
504,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0,6.44,large
505,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0,54.29,large
506,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0,11.16,large
507,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0,0,small


In [13]:
df['size_category'].value_counts()

small    371
large    138
Name: size_category, dtype: int64

### NOTE
* There is class imbalance, for the given dataset.
* We will be using SMOTE technique to balance out both target classes.
* However to use SMOTE technique, we first have to convert categorical features in numerical ones.
* We will be using Label Encoding for Days & Months columns 

In [14]:
from sklearn.preprocessing import LabelEncoder
labels = LabelEncoder()
enc_month = labels.fit_transform(df['month'])
enc_df_month = pd.DataFrame(enc_month,columns=['enc_month'])
enc_df_month

,enc_month
0,7
1,10
2,10
3,7
4,7
...,...
504,1
505,1
506,1
507,1


In [15]:
enc_day = labels.fit_transform(df['day'])
enc_df_day = pd.DataFrame(enc_day,columns=['enc_day'])
enc_df_day

,enc_day
0,0
1,5
2,2
3,0
4,3
...,...
504,3
505,3
506,3
507,2


In [16]:
enc_size = labels.fit_transform(df['size_category'])
enc_df_size = pd.DataFrame(enc_size,columns=['size_category'])
enc_df_size

,size_category
0,1
1,1
2,1
3,1
4,1
...,...
504,0
505,0
506,0
507,1


In [17]:
enc_concate = pd.concat((enc_df_month,enc_df_day), axis=1)
enc_concate

,enc_month,enc_day
0,7,0
1,10,5
2,10,2
3,7,0
4,7,3
...,...,...
504,1,3
505,1,3
506,1,3
507,1,2


In [18]:
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...
504,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0,6.44,large
505,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0,54.29,large
506,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0,11.16,large
507,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0,0,small


In [19]:
df = df.drop(columns=['month','day','size_category'],axis=1)
df

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0,0
...,...,...,...,...,...,...,...,...,...
504,81.6,56.7,665.6,1.9,27.8,32,2.7,0,6.44
505,81.6,56.7,665.6,1.9,21.9,71,5.8,0,54.29
506,81.6,56.7,665.6,1.9,21.2,70,6.7,0,11.16
507,94.4,146.0,614.7,11.3,25.6,42,4.0,0,0


In [20]:
enc_df = pd.concat((enc_concate,df,enc_df_size),axis=1)
enc_df

,enc_month,enc_day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0,1
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0,0,1
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0,1
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0,1
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
504,1,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0,6.44,0
505,1,3,81.6,56.7,665.6,1.9,21.9,71,5.8,0,54.29,0
506,1,3,81.6,56.7,665.6,1.9,21.2,70,6.7,0,11.16,0
507,1,2,94.4,146.0,614.7,11.3,25.6,42,4.0,0,0,1


#### Now we have dataframe in numerical format exactly how we want

In [21]:
X = enc_df.drop(columns='size_category',axis=1)
Y = enc_df['size_category']

In [22]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X,Y = oversample.fit_resample(X,Y)

In [23]:
final_df = pd.concat((X,Y),axis=1)
final_df

,enc_month,enc_day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,7,0,86.200000,26.200000,94.300000,5.100000,8.200000,51,6.700000,0,0,1
1,10,5,90.600000,35.400000,669.100000,6.700000,18.000000,33,0.900000,0,0,1
2,10,2,90.600000,43.700000,686.900000,6.700000,14.600000,33,1.300000,0,0,1
3,7,0,91.700000,33.300000,77.500000,9.000000,8.300000,97,4.000000,0.2,0,1
4,7,3,89.300000,51.300000,102.200000,9.600000,11.400000,99,1.800000,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
737,11,1,91.243425,125.225071,797.008042,8.355251,17.707947,40,2.200000,0,13.6764,0
738,1,2,93.663518,232.559263,718.018525,9.020187,22.614998,57,4.328334,0,9.68051,0
739,11,2,91.645671,122.895862,792.013179,8.221456,20.775785,29,5.252069,0,20.8543,0
740,11,2,91.408287,99.077913,746.639251,7.534639,23.758114,28,2.934293,0,6.48058,0


In [24]:
final_df['size_category'].value_counts()

1    371
0    371
Name: size_category, dtype: int64

In [25]:
final_df.duplicated().sum()

0

In [26]:
final_df = pd.get_dummies(final_df,columns=['enc_month','enc_day'])
final_df

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category,...,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
0,86.200000,26.200000,94.300000,5.100000,8.200000,51,6.700000,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,90.600000,35.400000,669.100000,6.700000,18.000000,33,0.900000,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2,90.600000,43.700000,686.900000,6.700000,14.600000,33,1.300000,0,0,1,...,0,1,0,0,0,1,0,0,0,0
3,91.700000,33.300000,77.500000,9.000000,8.300000,97,4.000000,0.2,0,1,...,0,0,0,1,0,0,0,0,0,0
4,89.300000,51.300000,102.200000,9.600000,11.400000,99,1.800000,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,91.243425,125.225071,797.008042,8.355251,17.707947,40,2.200000,0,13.6764,0,...,0,0,1,0,1,0,0,0,0,0
738,93.663518,232.559263,718.018525,9.020187,22.614998,57,4.328334,0,9.68051,0,...,0,0,0,0,0,1,0,0,0,0
739,91.645671,122.895862,792.013179,8.221456,20.775785,29,5.252069,0,20.8543,0,...,0,0,1,0,0,1,0,0,0,0
740,91.408287,99.077913,746.639251,7.534639,23.758114,28,2.934293,0,6.48058,0,...,0,0,1,0,0,1,0,0,0,0


In [27]:
X = final_df.drop(columns=['size_category'])
Y = final_df['size_category']

In [28]:
X

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,enc_month_0,...,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
0,86.200000,26.200000,94.300000,5.100000,8.200000,51,6.700000,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,90.600000,35.400000,669.100000,6.700000,18.000000,33,0.900000,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,90.600000,43.700000,686.900000,6.700000,14.600000,33,1.300000,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,91.700000,33.300000,77.500000,9.000000,8.300000,97,4.000000,0.2,0,0,...,0,0,0,1,0,0,0,0,0,0
4,89.300000,51.300000,102.200000,9.600000,11.400000,99,1.800000,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,91.243425,125.225071,797.008042,8.355251,17.707947,40,2.200000,0,13.6764,0,...,0,0,1,0,1,0,0,0,0,0
738,93.663518,232.559263,718.018525,9.020187,22.614998,57,4.328334,0,9.68051,0,...,0,0,0,0,0,1,0,0,0,0
739,91.645671,122.895862,792.013179,8.221456,20.775785,29,5.252069,0,20.8543,0,...,0,0,1,0,0,1,0,0,0,0
740,91.408287,99.077913,746.639251,7.534639,23.758114,28,2.934293,0,6.48058,0,...,0,0,1,0,0,1,0,0,0,0


## Splitting data into train,validation & test

In [29]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test = train_test_split(X,Y,test_size=0.2,random_state=42, stratify=Y)

In [30]:
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.25,random_state=42,stratify=y)

In [31]:
len(x_train),len(x_val),len(x_test)

(444, 149, 149)

In [32]:
len(y_train),len(y_val),len(y_test)

(444, 149, 149)

## Standardizating Data

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [34]:
x_train.iloc[:,:9]

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
296,53.400000,71.000000,233.800000,0.400000,10.600000,90,2.700000,0,0
654,93.618528,98.495999,543.207438,14.295005,28.164125,32,3.944415,0,8.82437
76,87.600000,52.200000,103.800000,5.000000,8.300000,72,3.100000,0,0
436,92.100000,207.000000,672.600000,8.200000,25.500000,29,1.800000,0,1.23
513,93.334298,188.853844,660.680453,15.287751,19.644931,75,4.900000,3.2392,10.2718
...,...,...,...,...,...,...,...,...,...
48,87.200000,23.900000,64.700000,4.100000,11.800000,35,1.800000,0,0
108,85.900000,19.500000,57.300000,2.800000,13.700000,43,5.800000,0,0
502,91.000000,166.900000,752.600000,7.100000,18.200000,62,5.400000,0,0.43
741,85.400000,25.400000,349.700000,2.600000,5.035489,23,8.500000,0,24.4165


In [35]:
x_val.iloc[:,:9]

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
195,92.900000,137.000000,706.400000,9.200000,21.500000,15,0.900000,0,11.06
196,91.000000,129.500000,692.600000,7.000000,13.900000,59,6.300000,0,11.24
729,85.400000,25.400000,349.700000,2.600000,4.763706,21,8.500000,0,20.1157
673,90.477202,39.917099,86.771503,7.577202,12.884974,47,2.145078,0,9.56736
324,92.200000,102.300000,751.500000,8.400000,19.700000,35,1.800000,0,0
...,...,...,...,...,...,...,...,...,...
538,94.156629,84.096406,690.603449,16.377902,20.840748,41,4.900000,0,25.4801
607,90.636057,48.621658,98.416858,7.818029,14.775486,60,3.441115,0,12.8019
45,94.300000,85.100000,692.300000,15.900000,25.900000,24,4.000000,0,0
77,90.100000,108.000000,529.800000,12.500000,14.700000,66,2.700000,0,0


In [36]:
scaler.fit(x_train.iloc[:,:9])

StandardScaler()

In [37]:
scaler.fit(x_val.iloc[:,:9])

StandardScaler()

In [38]:
scaled_x = scaler.transform(x_train.iloc[:,:9])
print(scaled_x)

[[-12.1062811   -0.58007536  -1.21941527 ...  -0.77442096  -0.09769031
   -0.45679719]
 [  0.79862233  -0.15427325   0.04170399 ...  -0.0900896   -0.09769031
   -0.12985291]
 [ -1.13254053  -0.87121153  -1.74928457 ...  -0.55445204  -0.09769031
   -0.45679719]
 ...
 [ -0.04158387   0.90502881   0.89517084 ...   0.71036925  -0.09769031
   -0.44086562]
 [ -1.83845367  -1.28623543  -0.7470164  ...   2.41512839  -0.09769031
    0.44783723]
 [ -1.83845367  -1.28623543  -0.7470164  ...   2.41512839  -0.09769031
    0.25061592]]


In [39]:
scaled_xv = scaler.transform(x_val.iloc[:,:9])
print(scaled_xv)

[[ 0.56806839  0.44199842  0.70686344 ... -1.76428111 -0.09769031
  -0.04702243]
 [-0.04158387  0.32585367  0.65061578 ...  1.20529932 -0.09769031
  -0.0403534 ]
 [-1.83845367 -1.28623543 -0.7470164  ...  2.41512839 -0.09769031
   0.28849218]
 ...
 [ 1.01728584 -0.36172323  0.649393   ... -0.05952197 -0.09769031
  -0.45679719]
 [-0.33036651 -0.0070946  -0.01294363 ... -0.77442096 -0.09769031
  -0.45679719]
 [ 0.11885094  0.33514525  1.11730836 ... -1.04938212 -0.09769031
   0.5876468 ]]


In [40]:
x_train.columns

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'enc_month_0', 'enc_month_1', 'enc_month_2', 'enc_month_3',
       'enc_month_4', 'enc_month_5', 'enc_month_6', 'enc_month_7',
       'enc_month_8', 'enc_month_9', 'enc_month_10', 'enc_month_11',
       'enc_day_0', 'enc_day_1', 'enc_day_2', 'enc_day_3', 'enc_day_4',
       'enc_day_5', 'enc_day_6'],
      dtype='object')

In [41]:
x_val.columns

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'enc_month_0', 'enc_month_1', 'enc_month_2', 'enc_month_3',
       'enc_month_4', 'enc_month_5', 'enc_month_6', 'enc_month_7',
       'enc_month_8', 'enc_month_9', 'enc_month_10', 'enc_month_11',
       'enc_day_0', 'enc_day_1', 'enc_day_2', 'enc_day_3', 'enc_day_4',
       'enc_day_5', 'enc_day_6'],
      dtype='object')

In [42]:
scaledvalue = pd.DataFrame(scaled_x,columns=['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area'],index=x_train.index)
scaledvalue

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
296,-12.106281,-0.580075,-1.219415,-2.212335,-1.288364,2.772263,-0.774421,-0.097690,-0.456797
654,0.798622,-0.154273,0.041704,1.262665,1.565278,-0.786545,-0.090090,-0.097690,-0.129853
76,-1.132541,-0.871212,-1.749285,-1.061922,-1.662044,1.667805,-0.554452,-0.097690,-0.456797
436,0.311373,1.526016,0.569097,-0.261634,1.132438,-0.970622,-1.269351,-0.097690,-0.411225
513,0.707421,1.245005,0.520514,1.510941,0.181165,1.851882,0.435408,41.391686,-0.076225
...,...,...,...,...,...,...,...,...,...
48,-1.260888,-1.309464,-1.908653,-1.287003,-1.093400,-0.602469,-1.269351,-0.097690,-0.456797
108,-1.678019,-1.377603,-1.938815,-1.612120,-0.784707,-0.111599,0.930338,-0.097690,-0.456797
502,-0.041584,0.905029,0.895171,-0.536733,-0.053592,1.054218,0.710369,-0.097690,-0.440866
741,-1.838454,-1.286235,-0.747016,-1.662138,-2.192429,-1.338774,2.415128,-0.097690,0.447837


In [43]:
scaledval = pd.DataFrame(scaled_xv,columns=['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area'],index=x_val.index)
scaledval

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
195,0.568068,0.441998,0.706863,-0.011544,0.482558,-1.829644,-1.764281,-0.09769,-0.047022
196,-0.041584,0.325854,0.650616,-0.561742,-0.752213,0.870141,1.205299,-0.09769,-0.040353
729,-1.838454,-1.286235,-0.747016,-1.662138,-2.236586,-1.461492,2.415128,-0.09769,0.288492
673,-0.209334,-1.061424,-1.818691,-0.417390,-0.917124,0.133836,-1.079585,-0.09769,-0.102325
324,0.343460,-0.095365,0.890687,-0.211616,0.190112,-0.602469,-1.269351,-0.09769,-0.456797
...,...,...,...,...,...,...,...,...,...
538,0.971283,-0.377265,0.642478,1.783576,0.375450,-0.234316,0.435408,-0.09769,0.487243
607,-0.158362,-0.926626,-1.771226,-0.357161,-0.609973,0.931500,-0.366866,-0.09769,0.017516
45,1.017286,-0.361723,0.649393,1.664058,1.197426,-1.277415,-0.059522,-0.09769,-0.456797
77,-0.330367,-0.007095,-0.012944,0.813752,-0.622237,1.299653,-0.774421,-0.09769,-0.456797


In [44]:
x_train = x_train.drop(columns=['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area'])

In [45]:
x_val = x_val.drop(columns=['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area'])

In [46]:
x_train

,enc_month_0,enc_month_1,enc_month_2,enc_month_3,enc_month_4,enc_month_5,enc_month_6,enc_month_7,enc_month_8,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
296,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
654,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
76,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
436,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
513,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
108,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
502,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
741,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [47]:
x_val

,enc_month_0,enc_month_1,enc_month_2,enc_month_3,enc_month_4,enc_month_5,enc_month_6,enc_month_7,enc_month_8,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
195,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
196,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
729,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
673,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
324,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
607,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
45,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
77,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [48]:
x_train  = pd.concat((scaledvalue,x_train),axis=1)
x_train

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,enc_month_0,...,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
296,-12.106281,-0.580075,-1.219415,-2.212335,-1.288364,2.772263,-0.774421,-0.097690,-0.456797,0,...,0,0,0,0,0,1,0,0,0,0
654,0.798622,-0.154273,0.041704,1.262665,1.565278,-0.786545,-0.090090,-0.097690,-0.129853,0,...,0,0,0,0,1,0,0,0,0,0
76,-1.132541,-0.871212,-1.749285,-1.061922,-1.662044,1.667805,-0.554452,-0.097690,-0.456797,0,...,0,0,0,0,1,0,0,0,0,0
436,0.311373,1.526016,0.569097,-0.261634,1.132438,-0.970622,-1.269351,-0.097690,-0.411225,0,...,0,0,0,0,1,0,0,0,0,0
513,0.707421,1.245005,0.520514,1.510941,0.181165,1.851882,0.435408,41.391686,-0.076225,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,-1.260888,-1.309464,-1.908653,-1.287003,-1.093400,-0.602469,-1.269351,-0.097690,-0.456797,0,...,0,0,0,0,1,0,0,0,0,0
108,-1.678019,-1.377603,-1.938815,-1.612120,-0.784707,-0.111599,0.930338,-0.097690,-0.456797,0,...,0,0,0,1,0,0,0,0,0,0
502,-0.041584,0.905029,0.895171,-0.536733,-0.053592,1.054218,0.710369,-0.097690,-0.440866,0,...,0,0,0,1,0,0,0,0,0,0
741,-1.838454,-1.286235,-0.747016,-1.662138,-2.192429,-1.338774,2.415128,-0.097690,0.447837,0,...,0,0,0,0,0,0,0,1,0,0


In [49]:
x_val  = pd.concat((scaledval,x_val),axis=1)
x_val

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,enc_month_0,...,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
195,0.568068,0.441998,0.706863,-0.011544,0.482558,-1.829644,-1.764281,-0.09769,-0.047022,0,...,0,0,1,0,0,0,0,1,0,0
196,-0.041584,0.325854,0.650616,-0.561742,-0.752213,0.870141,1.205299,-0.09769,-0.040353,0,...,0,0,1,0,0,0,0,0,1,0
729,-1.838454,-1.286235,-0.747016,-1.662138,-2.236586,-1.461492,2.415128,-0.09769,0.288492,0,...,0,0,0,0,0,1,0,0,0,0
673,-0.209334,-1.061424,-1.818691,-0.417390,-0.917124,0.133836,-1.079585,-0.09769,-0.102325,0,...,0,0,0,0,0,1,0,0,0,0
324,0.343460,-0.095365,0.890687,-0.211616,0.190112,-0.602469,-1.269351,-0.09769,-0.456797,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,0.971283,-0.377265,0.642478,1.783576,0.375450,-0.234316,0.435408,-0.09769,0.487243,0,...,0,0,1,1,0,0,0,0,0,0
607,-0.158362,-0.926626,-1.771226,-0.357161,-0.609973,0.931500,-0.366866,-0.09769,0.017516,0,...,0,0,0,0,0,1,0,0,0,0
45,1.017286,-0.361723,0.649393,1.664058,1.197426,-1.277415,-0.059522,-0.09769,-0.456797,0,...,0,0,1,0,0,0,0,0,0,1
77,-0.330367,-0.007095,-0.012944,0.813752,-0.622237,1.299653,-0.774421,-0.09769,-0.456797,0,...,0,0,0,1,0,0,0,0,0,0


In [50]:
x_train = x_train.astype('float64')
x_train

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,enc_month_0,...,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
296,-12.106281,-0.580075,-1.219415,-2.212335,-1.288364,2.772263,-0.774421,-0.097690,-0.456797,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
654,0.798622,-0.154273,0.041704,1.262665,1.565278,-0.786545,-0.090090,-0.097690,-0.129853,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
76,-1.132541,-0.871212,-1.749285,-1.061922,-1.662044,1.667805,-0.554452,-0.097690,-0.456797,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
436,0.311373,1.526016,0.569097,-0.261634,1.132438,-0.970622,-1.269351,-0.097690,-0.411225,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
513,0.707421,1.245005,0.520514,1.510941,0.181165,1.851882,0.435408,41.391686,-0.076225,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,-1.260888,-1.309464,-1.908653,-1.287003,-1.093400,-0.602469,-1.269351,-0.097690,-0.456797,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
108,-1.678019,-1.377603,-1.938815,-1.612120,-0.784707,-0.111599,0.930338,-0.097690,-0.456797,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
502,-0.041584,0.905029,0.895171,-0.536733,-0.053592,1.054218,0.710369,-0.097690,-0.440866,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
741,-1.838454,-1.286235,-0.747016,-1.662138,-2.192429,-1.338774,2.415128,-0.097690,0.447837,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [51]:
x_val = x_val.astype('float64')
x_val

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,enc_month_0,...,enc_month_9,enc_month_10,enc_month_11,enc_day_0,enc_day_1,enc_day_2,enc_day_3,enc_day_4,enc_day_5,enc_day_6
195,0.568068,0.441998,0.706863,-0.011544,0.482558,-1.829644,-1.764281,-0.09769,-0.047022,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
196,-0.041584,0.325854,0.650616,-0.561742,-0.752213,0.870141,1.205299,-0.09769,-0.040353,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
729,-1.838454,-1.286235,-0.747016,-1.662138,-2.236586,-1.461492,2.415128,-0.09769,0.288492,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
673,-0.209334,-1.061424,-1.818691,-0.417390,-0.917124,0.133836,-1.079585,-0.09769,-0.102325,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
324,0.343460,-0.095365,0.890687,-0.211616,0.190112,-0.602469,-1.269351,-0.09769,-0.456797,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,0.971283,-0.377265,0.642478,1.783576,0.375450,-0.234316,0.435408,-0.09769,0.487243,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
607,-0.158362,-0.926626,-1.771226,-0.357161,-0.609973,0.931500,-0.366866,-0.09769,0.017516,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
45,1.017286,-0.361723,0.649393,1.664058,1.197426,-1.277415,-0.059522,-0.09769,-0.456797,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
77,-0.330367,-0.007095,-0.012944,0.813752,-0.622237,1.299653,-0.774421,-0.09769,-0.456797,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model training

In [52]:
from sklearn.svm import SVC
Base_Model = SVC(class_weight='balanced')

In [53]:
Base_Model.fit(x_train,y_train)

SVC(class_weight='balanced')

In [54]:
y_pred = Base_Model.predict(x_val)
y_pred

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0])

In [55]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_val,y_pred)*100
score

73.8255033557047

In [56]:
from sklearn.model_selection import cross_val_score
score1 = cross_val_score(Base_Model,x,y,cv=5)
score1

array([0.61344538, 0.61344538, 0.55462185, 0.59322034, 0.62711864])

In [57]:
score1.mean()*100

60.037031761857286

## Hyperarameter Tuning

In [58]:
Base_Model.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': 'balanced',
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [59]:
random_params = dict(C= [1.0,2.0],
 degree= [3,5,7,9,11,13,15,17],
 gamma= ['scale','auto'],
 kernel= ['poly','sigmoid','rbf','linear'])

In [60]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
random = RandomizedSearchCV(Base_Model,
                            param_distributions= random_params,
                            cv=10,
                            verbose=3,
                            random_state=42)
random

RandomizedSearchCV(cv=10, estimator=SVC(class_weight='balanced'),
                   param_distributions={'C': [1.0, 2.0],
                                        'degree': [3, 5, 7, 9, 11, 13, 15, 17],
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['poly', 'sigmoid', 'rbf',
                                                   'linear']},
                   random_state=42, verbose=3)

In [61]:
random.fit(x_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=0.978 total time=   0.0s
[CV 2/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=0.978 total time=   0.0s
[CV 4/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=0.978 total time=   0.0s
[CV 5/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 6/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 7/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 8/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 9/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=0.955 total time=   0.0s
[CV 10/10] END C=1.0, degree=15, gamma=auto, kernel=linear;, score=0.977 total time=   0.0s
[CV 1/10] END C=1.0, degre

[CV 6/10] END C=1.0, degree=9, gamma=scale, kernel=rbf;, score=0.773 total time=   0.0s
[CV 7/10] END C=1.0, degree=9, gamma=scale, kernel=rbf;, score=0.705 total time=   0.0s
[CV 8/10] END C=1.0, degree=9, gamma=scale, kernel=rbf;, score=0.795 total time=   0.0s
[CV 9/10] END C=1.0, degree=9, gamma=scale, kernel=rbf;, score=0.705 total time=   0.0s
[CV 10/10] END C=1.0, degree=9, gamma=scale, kernel=rbf;, score=0.727 total time=   0.0s


RandomizedSearchCV(cv=10, estimator=SVC(class_weight='balanced'),
                   param_distributions={'C': [1.0, 2.0],
                                        'degree': [3, 5, 7, 9, 11, 13, 15, 17],
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['poly', 'sigmoid', 'rbf',
                                                   'linear']},
                   random_state=42, verbose=3)

In [62]:
random.best_params_

{'kernel': 'linear', 'gamma': 'auto', 'degree': 15, 'C': 1.0}

In [63]:
grid_params = dict(C= [1.0,1.1,1.2,1.3],
 degree= [11,13,15,17,19,21,23,25],
 gamma= ['scale','auto'],
 kernel= ['poly','rbf','linear'])

In [64]:
grid = GridSearchCV(Base_Model,
                    param_grid= grid_params,
                    cv=10,
                    verbose=3)
grid

GridSearchCV(cv=10, estimator=SVC(class_weight='balanced'),
             param_grid={'C': [1.0, 1.1, 1.2, 1.3],
                         'degree': [11, 13, 15, 17, 19, 21, 23, 25],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'rbf', 'linear']},
             verbose=3)

In [65]:
grid.fit(x_train,y_train)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits
[CV 1/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.511 total time=   0.0s
[CV 2/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.511 total time=   0.0s
[CV 3/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.578 total time=   0.0s
[CV 4/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.489 total time=   0.0s
[CV 5/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 6/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 7/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 8/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.500 total time=   0.0s
[CV 9/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 10/10] END C=1.0, degree=11, gamma=scale, kernel=poly;, score=0.500 total time=   0.0s
[CV 1/10] END C=1.0, degree=11, ga

[CV 5/10] END C=1.0, degree=13, gamma=auto, kernel=poly;, score=0.545 total time=   0.0s
[CV 6/10] END C=1.0, degree=13, gamma=auto, kernel=poly;, score=0.545 total time=   0.0s
[CV 7/10] END C=1.0, degree=13, gamma=auto, kernel=poly;, score=0.545 total time=   0.0s
[CV 8/10] END C=1.0, degree=13, gamma=auto, kernel=poly;, score=0.545 total time=   0.0s
[CV 9/10] END C=1.0, degree=13, gamma=auto, kernel=poly;, score=0.523 total time=   0.0s
[CV 10/10] END C=1.0, degree=13, gamma=auto, kernel=poly;, score=0.477 total time=   0.0s
[CV 1/10] END C=1.0, degree=13, gamma=auto, kernel=rbf;, score=0.800 total time=   0.0s
[CV 2/10] END C=1.0, degree=13, gamma=auto, kernel=rbf;, score=0.822 total time=   0.0s
[CV 3/10] END C=1.0, degree=13, gamma=auto, kernel=rbf;, score=0.867 total time=   0.0s
[CV 4/10] END C=1.0, degree=13, gamma=auto, kernel=rbf;, score=0.756 total time=   0.0s
[CV 5/10] END C=1.0, degree=13, gamma=auto, kernel=rbf;, score=0.886 total time=   0.0s
[CV 6/10] END C=1.0, degr

[CV 7/10] END C=1.0, degree=17, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 8/10] END C=1.0, degree=17, gamma=scale, kernel=poly;, score=0.500 total time=   0.0s
[CV 9/10] END C=1.0, degree=17, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 10/10] END C=1.0, degree=17, gamma=scale, kernel=poly;, score=0.500 total time=   0.0s
[CV 1/10] END C=1.0, degree=17, gamma=scale, kernel=rbf;, score=0.778 total time=   0.0s
[CV 2/10] END C=1.0, degree=17, gamma=scale, kernel=rbf;, score=0.756 total time=   0.0s
[CV 3/10] END C=1.0, degree=17, gamma=scale, kernel=rbf;, score=0.822 total time=   0.0s
[CV 4/10] END C=1.0, degree=17, gamma=scale, kernel=rbf;, score=0.622 total time=   0.0s
[CV 5/10] END C=1.0, degree=17, gamma=scale, kernel=rbf;, score=0.773 total time=   0.0s
[CV 6/10] END C=1.0, degree=17, gamma=scale, kernel=rbf;, score=0.773 total time=   0.0s
[CV 7/10] END C=1.0, degree=17, gamma=scale, kernel=rbf;, score=0.705 total time=   0.0s
[CV 8/10] END C=

[CV 1/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.800 total time=   0.0s
[CV 2/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.822 total time=   0.0s
[CV 3/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.867 total time=   0.0s
[CV 4/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.756 total time=   0.0s
[CV 5/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.886 total time=   0.0s
[CV 6/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.818 total time=   0.0s
[CV 7/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.818 total time=   0.0s
[CV 8/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.841 total time=   0.0s
[CV 9/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.773 total time=   0.0s
[CV 10/10] END C=1.0, degree=19, gamma=auto, kernel=rbf;, score=0.727 total time=   0.0s
[CV 1/10] END C=1.0, degree=19, gamma=auto, kernel=linear;, score=0.978 total time=   0.0s
[CV 2/10] END C=1.0, degree=

[CV 5/10] END C=1.0, degree=23, gamma=scale, kernel=rbf;, score=0.773 total time=   0.0s
[CV 6/10] END C=1.0, degree=23, gamma=scale, kernel=rbf;, score=0.773 total time=   0.0s
[CV 7/10] END C=1.0, degree=23, gamma=scale, kernel=rbf;, score=0.705 total time=   0.0s
[CV 8/10] END C=1.0, degree=23, gamma=scale, kernel=rbf;, score=0.795 total time=   0.0s
[CV 9/10] END C=1.0, degree=23, gamma=scale, kernel=rbf;, score=0.705 total time=   0.0s
[CV 10/10] END C=1.0, degree=23, gamma=scale, kernel=rbf;, score=0.727 total time=   0.0s
[CV 1/10] END C=1.0, degree=23, gamma=scale, kernel=linear;, score=0.978 total time=   0.0s
[CV 2/10] END C=1.0, degree=23, gamma=scale, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/10] END C=1.0, degree=23, gamma=scale, kernel=linear;, score=0.978 total time=   0.0s
[CV 4/10] END C=1.0, degree=23, gamma=scale, kernel=linear;, score=0.978 total time=   0.0s
[CV 5/10] END C=1.0, degree=23, gamma=scale, kernel=linear;, score=1.000 total time=   0.0s
[CV 6

...

[CV 5/10] END C=1.3, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 6/10] END C=1.3, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 7/10] END C=1.3, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 8/10] END C=1.3, degree=15, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 9/10] END C=1.3, degree=15, gamma=auto, kernel=linear;, score=0.955 total time=   0.0s
[CV 10/10] END C=1.3, degree=15, gamma=auto, kernel=linear;, score=0.977 total time=   0.0s
[CV 1/10] END C=1.3, degree=17, gamma=scale, kernel=poly;, score=0.511 total time=   0.0s
[CV 2/10] END C=1.3, degree=17, gamma=scale, kernel=poly;, score=0.511 total time=   0.0s
[CV 3/10] END C=1.3, degree=17, gamma=scale, kernel=poly;, score=0.578 total time=   0.0s
[CV 4/10] END C=1.3, degree=17, gamma=scale, kernel=poly;, score=0.489 total time=   0.0s
[CV 5/10] END C=1.3, degree=17, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV

[CV 4/10] END C=1.3, degree=19, gamma=auto, kernel=poly;, score=0.489 total time=   0.0s
[CV 5/10] END C=1.3, degree=19, gamma=auto, kernel=poly;, score=0.523 total time=   0.0s
[CV 6/10] END C=1.3, degree=19, gamma=auto, kernel=poly;, score=0.523 total time=   0.0s
[CV 7/10] END C=1.3, degree=19, gamma=auto, kernel=poly;, score=0.545 total time=   0.0s
[CV 8/10] END C=1.3, degree=19, gamma=auto, kernel=poly;, score=0.500 total time=   0.0s
[CV 9/10] END C=1.3, degree=19, gamma=auto, kernel=poly;, score=0.523 total time=   0.0s
[CV 10/10] END C=1.3, degree=19, gamma=auto, kernel=poly;, score=0.477 total time=   0.0s
[CV 1/10] END C=1.3, degree=19, gamma=auto, kernel=rbf;, score=0.822 total time=   0.0s
[CV 2/10] END C=1.3, degree=19, gamma=auto, kernel=rbf;, score=0.867 total time=   0.0s
[CV 3/10] END C=1.3, degree=19, gamma=auto, kernel=rbf;, score=0.911 total time=   0.0s
[CV 4/10] END C=1.3, degree=19, gamma=auto, kernel=rbf;, score=0.778 total time=   0.0s
[CV 5/10] END C=1.3, deg

[CV 7/10] END C=1.3, degree=23, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 8/10] END C=1.3, degree=23, gamma=scale, kernel=poly;, score=0.500 total time=   0.0s
[CV 9/10] END C=1.3, degree=23, gamma=scale, kernel=poly;, score=0.523 total time=   0.0s
[CV 10/10] END C=1.3, degree=23, gamma=scale, kernel=poly;, score=0.500 total time=   0.0s
[CV 1/10] END C=1.3, degree=23, gamma=scale, kernel=rbf;, score=0.822 total time=   0.0s
[CV 2/10] END C=1.3, degree=23, gamma=scale, kernel=rbf;, score=0.756 total time=   0.0s
[CV 3/10] END C=1.3, degree=23, gamma=scale, kernel=rbf;, score=0.867 total time=   0.0s
[CV 4/10] END C=1.3, degree=23, gamma=scale, kernel=rbf;, score=0.667 total time=   0.0s
[CV 5/10] END C=1.3, degree=23, gamma=scale, kernel=rbf;, score=0.795 total time=   0.0s
[CV 6/10] END C=1.3, degree=23, gamma=scale, kernel=rbf;, score=0.795 total time=   0.0s
[CV 7/10] END C=1.3, degree=23, gamma=scale, kernel=rbf;, score=0.795 total time=   0.0s
[CV 8/10] END C=

[CV 5/10] END C=1.3, degree=25, gamma=auto, kernel=rbf;, score=0.955 total time=   0.0s
[CV 6/10] END C=1.3, degree=25, gamma=auto, kernel=rbf;, score=0.909 total time=   0.0s
[CV 7/10] END C=1.3, degree=25, gamma=auto, kernel=rbf;, score=0.841 total time=   0.0s
[CV 8/10] END C=1.3, degree=25, gamma=auto, kernel=rbf;, score=0.818 total time=   0.0s
[CV 9/10] END C=1.3, degree=25, gamma=auto, kernel=rbf;, score=0.818 total time=   0.0s
[CV 10/10] END C=1.3, degree=25, gamma=auto, kernel=rbf;, score=0.795 total time=   0.0s
[CV 1/10] END C=1.3, degree=25, gamma=auto, kernel=linear;, score=0.978 total time=   0.0s
[CV 2/10] END C=1.3, degree=25, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 3/10] END C=1.3, degree=25, gamma=auto, kernel=linear;, score=0.978 total time=   0.0s
[CV 4/10] END C=1.3, degree=25, gamma=auto, kernel=linear;, score=0.978 total time=   0.0s
[CV 5/10] END C=1.3, degree=25, gamma=auto, kernel=linear;, score=1.000 total time=   0.0s
[CV 6/10] END C=

GridSearchCV(cv=10, estimator=SVC(class_weight='balanced'),
             param_grid={'C': [1.0, 1.1, 1.2, 1.3],
                         'degree': [11, 13, 15, 17, 19, 21, 23, 25],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'rbf', 'linear']},
             verbose=3)

In [66]:
grid.best_params_

{'C': 1.0, 'degree': 11, 'gamma': 'scale', 'kernel': 'linear'}

In [67]:
Final_Model = SVC(C=1.0,
                  kernel='linear',
                  degree=11,
                  gamma='scale', class_weight='balanced')

In [68]:
Final_Model.fit(x_train,y_train)

SVC(class_weight='balanced', degree=11, kernel='linear')

In [69]:
y_predf = Final_Model.predict(x_val)
y_predf

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0])

In [70]:
from sklearn.metrics import classification_report
report = classification_report(y_val,y_predf)
print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        74
           1       1.00      0.96      0.98        75

    accuracy                           0.98       149
   macro avg       0.98      0.98      0.98       149
weighted avg       0.98      0.98      0.98       149



In [71]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_val,y_predf)
print(matrix)

[[74  0]
 [ 3 72]]


In [72]:
from sklearn.model_selection import cross_val_score
score1 = cross_val_score(Final_Model,x,y,cv=5)
score1

array([0.97478992, 0.99159664, 0.98319328, 0.98305085, 0.99152542])

In [73]:
score1.mean()*100

98.48312206238428